In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(reshape2)
library(ggsci)
library(rstatix)
library(grid)
library(RColorBrewer)
library(hise)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




# Read Mean Expression Data

In [2]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA_meta_data-2024-05-09.csv")

In [3]:
meta_data$Ages<-as.numeric(substr(meta_data$sample.drawDate,1,4))-meta_data$subject.birthYear

In [4]:
meta_data_subset<-meta_data %>% filter(Covid_exclusion=='no',
                                       !sample.visitName%in% c('Flu Year 1 Day 7',
                                                               'Flu Year 2 Day 7'))%>% 
                                filter(sample.daysSinceFirstVisit<600)

In [5]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/", meta_data_subset$pbmc_sample_id, ".csv")

In [6]:
df_list<-read_pseudobulk_expression(file_list, mc_cores = 60)

[1] "Total reading time: 54.649 seconds"
[1] "The length of the list matches the length of the input path."


# Read Deseq2 Result

In [7]:
deseq2_result_Y1D0<-read.csv('/home/jupyter/BRI_Figures_Final_V2/Figure2/02_DEG/Deseq2_Result_Y1D0.csv')

In [8]:
# filter significant genes 
deseq2_result_Y1D0_AgeGroup_sig<-deseq2_result_Y1D0 %>% 
  filter(contrast == "cohort.cohortGuid") %>%
  filter(padj < 0.05, 
         abs(log2FoldChange) > 0.1)

# Filter Genes and CellTypes

In [9]:
df_degs_counts<-as.data.frame(table(deseq2_result_Y1D0_AgeGroup_sig$celltype,
                                    deseq2_result_Y1D0_AgeGroup_sig$Direction))

In [12]:
df_degs_counts_filtered<-df_degs_counts %>% filter(Var2=="HigherInBR2") %>% arrange(Var1)   %>% filter(Freq>20)

# Construct Composite Score

In [12]:
score_df_list <- mclapply(unique(df_degs_counts_filtered$Var1), function(celltype_single) {
  #get mean expression of DEG genes
  top_n <- 100
  Direction_of_DEG <- "HigherInBR2"
  
  selected_genes <- deseq2_result_Y1D0_AgeGroup_sig %>%
    filter(celltype == celltype_single, Direction == Direction_of_DEG) %>% 
    arrange(padj, desc(abs(log2FoldChange))) %>% 
    slice_head(n = top_n) %>%
    select(gene) %>%
    pull()
  
  long_format <- filter_genes_and_celltype(df_list, 
                                           selected_genes, 
                                           celltype_single, 
                                           longformat = TRUE)
  long_format <- left_join(long_format, meta_data)
  
  long_format <- long_format %>%
    group_by(Gene) %>%
    mutate(Z_score_Mean_Expression = scale(Mean_Expression)) %>%
    ungroup()
    
 
  #construct compostite score by sum of z scored mean expression
  composite_score_df <- long_format %>%
    group_by(pbmc_sample_id) %>%
    summarise(composite_score_sum_mean = sum(Mean_Expression, na.rm = TRUE),
              composite_score_sum_scaled_mean = sum(Z_score_Mean_Expression, na.rm = TRUE)) %>%
    as.data.frame()
  
  composite_score_df <- left_join(composite_score_df, meta_data)
  composite_score_df$celltype <- celltype_single
  
  return(composite_score_df)
}, mc.cores = 10)

In [13]:
composite_score_df_all<-do.call(rbind,score_df_list)

In [14]:
composite_score_df_all<-composite_score_df_all %>% 
                        mutate(AgeGroup=case_when(cohort.cohortGuid=="BR1"~"Young", TRUE~'Older'))

In [15]:
write.csv(composite_score_df_all,'BRI_CompositeScore_Up.csv')

In [16]:
score_df_list <- mclapply(unique(df_degs_counts_filtered$Var1), function(celltype_single) {
  #get mean expression of DEG genes
  top_n <- 100
  Direction_of_DEG <- "HigherInBR1"
  
  selected_genes <- deseq2_result_Y1D0_AgeGroup_sig %>%
    filter(celltype == celltype_single, Direction == Direction_of_DEG) %>% 
    arrange(padj, desc(abs(log2FoldChange))) %>% 
    slice_head(n = top_n) %>%
    select(gene) %>%
    pull()
  
  long_format <- filter_genes_and_celltype(df_list, 
                                           selected_genes, 
                                           celltype_single, 
                                           longformat = TRUE)
  long_format <- left_join(long_format, meta_data)
  
  long_format <- long_format %>%
    group_by(Gene) %>%
    mutate(Z_score_Mean_Expression = scale(Mean_Expression)) %>%
    ungroup()
    
 
  #construct compostite score by sum of z scored mean expression
  composite_score_df <- long_format %>%
    group_by(pbmc_sample_id) %>%
    summarise(composite_score_sum_mean = sum(Mean_Expression, na.rm = TRUE),
              composite_score_sum_scaled_mean = sum(Z_score_Mean_Expression, na.rm = TRUE)) %>%
    as.data.frame()
  
  composite_score_df <- left_join(composite_score_df, meta_data)
  composite_score_df$celltype <- celltype_single
  
  return(composite_score_df)
}, mc.cores = 10)

In [17]:
composite_score_df_all<-do.call(rbind,score_df_list)

In [18]:
composite_score_df_all<-composite_score_df_all %>% 
                        mutate(AgeGroup=case_when(cohort.cohortGuid=="BR1"~"Young", TRUE~'Older'))

In [19]:
write.csv(composite_score_df_all,'BRI_CompositeScore_Down.csv')